In [16]:
import glob
import json

dicts_list = []

for fp in glob.glob('../*.json'):
    try:
        with open(fp, 'r', encoding='utf-8') as f:
            dicts_list.append(json.load(f))
            uuid = f.name
            uuid = uuid.removeprefix("../sim_dict_")
            uuid = uuid.removesuffix(".json")
            dicts_list[-1]['uuid'] = uuid
    except json.JSONDecodeError:
        print(f"Warning: failed to parse {fp}")

In [17]:
len(dicts_list)

4207

In [18]:
new = []
fail =[]
for i in dicts_list:
    dd = i['progress_dict']
    go = dd[list(dd).pop()][0] =='coreloss_project'
    # print(dd[list(dd).pop()][0])
    if go:
        new.append(i)
    else:
        fail.append(i)
len(new)

986

In [19]:
len(fail)

3221

In [20]:
import pandas as pd
df = pd.DataFrame(new)
df.head()
variables = df['v']
data = df['data']
data.head()

0    {'simulation parameter': {'A [mm]': {'0': 84.8...
1    {'simulation parameter': {'A [mm]': {'0': 50},...
2    {'simulation parameter': {'A [mm]': {'0': 58.4...
3    {'simulation parameter': {'A [mm]': {'0': 84.8...
4    {'simulation parameter': {'A [mm]': {'0': 53.6...
Name: data, dtype: object

In [21]:
import pandas as pd
df = pd.DataFrame(new)
df.head()
variables = df['v']
data = df['data']


variables_df = pd.DataFrame(variables.to_list())
data_df = pd.DataFrame(data.to_list())
data_df = data_df.drop(columns="simulation parameter")
data_df = data_df.drop(columns="_get_copper_loss_parameter")
data_df = data_df.drop(columns="coreloss parameter")
data_df = data_df.drop(columns="B mean report")
df_new = df[['uuid']]
df_new = df_new.join(variables_df)
df_new = df_new.join(data_df)
magnetic = pd.DataFrame(df_new.pop('_get_magnetic_report').to_list())
magnetic.columns = ['Ltx_uH', 'Lrx1_uH', 'Lrx2_uH', 'M1', 'M2', 'k1', 'k2',
       'Lmt_uH', 'Lmr1_uH', 'Lmr2_uH', 'Llt_uH', 'Llr1_uH', 'Llr2_uH']
df_new = df_new.join(magnetic)
df_new.head()

,uuid,w1,l1_leg,l1_top,l1_center,l2,h1,l2_tap,ratio,Tx_turns,...,M1,M2,k1,k2,Lmt_uH,Lmr1_uH,Lmr2_uH,Llt_uH,Llr1_uH,Llr2_uH
0,6f06d2a2-6ed5-4c32-86a3-15ab68ea1040,10.4,10.6,4.6,18.8,22.4,34.0,0.0,0.51,14.0,...,49.632585,49.633399,0.982330,0.982138,241.932409,9.825494,9.825816,8.782068,0.356662,0.360646
1,f4293267-75da-4a1c-a66f-3549d607ac63,17.6,8.2,3.4,22.4,5.6,19.6,0.0,0.50,14.0,...,89.182610,89.182436,0.988417,0.988243,437.976426,17.741479,17.741409,10.325444,0.418261,0.424662
2,54d1a301-d86f-4aa2-91ed-c0634f8c11ad,26.0,4.6,3.4,21.2,14.0,43.6,0.0,0.51,14.0,...,86.656572,86.657189,0.990480,0.990426,427.079881,17.249842,17.250088,8.249404,0.333195,0.335122
3,68e1736f-ee54-4cb1-bf86-0618cfa951f3,20.0,8.2,2.2,21.2,23.6,26.8,0.0,0.50,14.0,...,78.793883,78.793745,0.983796,0.983698,384.667485,15.621023,15.620968,12.776150,0.518829,0.522037
4,e151eee3-84f9-43cf-a173-5fd9dfbc54e8,24.8,9.4,2.2,9.2,12.8,28.0,0.0,0.51,14.0,...,75.263125,75.261360,0.986038,0.985833,368.004946,14.965738,14.965036,10.495474,0.426822,0.433194


In [22]:
copper_loss =pd.DataFrame(df_new.pop("copper loss data").to_list())
to_drop = []
for k in copper_loss.columns:
    if not 'P_'in k:
        to_drop.append(k)
    

copper_loss = copper_loss.drop(columns=to_drop).map(lambda x: x['0'])
copper_loss.columns = ['Copperloss_Tx','Copperloss_Rx_1', 'Copperloss_Rx_2']
copper_loss

,Copperloss_Tx,Copperloss_Rx_1,Copperloss_Rx_2
0,18.882762,2.631196,0.908127
1,10.987136,4.986744,1.218263
2,15.518811,1.564721,0.395008
3,5.231373,2.197519,0.327247
4,8.128260,4.213384,0.633982
...,...,...,...
981,8.551044,9.497925,0.992643
982,5.482404,3.992082,0.943417
983,22.119206,3.594430,0.802143
984,6.195132,3.594031,0.896810


In [23]:
df_new = df_new.join(copper_loss)

df_new.keys()

Index(['uuid', 'w1', 'l1_leg', 'l1_top', 'l1_center', 'l2', 'h1', 'l2_tap',
       'ratio', 'Tx_turns', 'Tx_tap', 'Tx_height', 'Tx_preg', 'Rx_turns',
       'Rx_tap', 'Rx_height', 'Rx_preg', 'g1', 'g2', 'Tx_space_x',
       'Tx_space_y', 'Rx_space_x', 'Rx_space_y', 'core_N_w1', 'core_P_w1',
       'Tx_layer_space_x', 'Tx_layer_space_y', 'Rx_layer_space_x',
       'Rx_layer_space_y', 'Tx_width', 'Rx_width', 'seed', 'A', 'B', 'C', 'D',
       'E', 'coreloss_project', '__get_B_field', 'Ltx_uH', 'Lrx1_uH',
       'Lrx2_uH', 'M1', 'M2', 'k1', 'k2', 'Lmt_uH', 'Lmr1_uH', 'Lmr2_uH',
       'Llt_uH', 'Llr1_uH', 'Llr2_uH', 'Copperloss_Tx', 'Copperloss_Rx_1',
       'Copperloss_Rx_2'],
      dtype='object')

In [24]:
coreloss = pd.DataFrame(df_new.pop('coreloss_project').to_list())

In [25]:
# 1) Coreloss [mW]→W, [kW]→W 로 변환해서 우선 채우고, 마지막에 [W] 로 채움
coreloss['Coreloss'] = (
    coreloss['Coreloss [W]']
      .fillna(coreloss.get('Coreloss [mW]', 0) * 1e-3)
      .fillna(coreloss.get('Coreloss [kW]', 0) * 1e3)
)

# 2) 이제 단위별 원본 컬럼들 삭제
coreloss.drop(columns=[c for c in coreloss.columns if c.startswith('Coreloss [')], inplace=True)
coreloss.columns = ['Coreloss_W']
coreloss

,Coreloss_W
0,9.10694
1,1.98987
2,2.72299
3,7.77997
4,3.85803
...,...
981,3.48804
982,3.12454
983,1.28705
984,2.87923


In [26]:
df_new = df_new.join(coreloss)

df_new.keys()

Index(['uuid', 'w1', 'l1_leg', 'l1_top', 'l1_center', 'l2', 'h1', 'l2_tap',
       'ratio', 'Tx_turns', 'Tx_tap', 'Tx_height', 'Tx_preg', 'Rx_turns',
       'Rx_tap', 'Rx_height', 'Rx_preg', 'g1', 'g2', 'Tx_space_x',
       'Tx_space_y', 'Rx_space_x', 'Rx_space_y', 'core_N_w1', 'core_P_w1',
       'Tx_layer_space_x', 'Tx_layer_space_y', 'Rx_layer_space_x',
       'Rx_layer_space_y', 'Tx_width', 'Rx_width', 'seed', 'A', 'B', 'C', 'D',
       'E', '__get_B_field', 'Ltx_uH', 'Lrx1_uH', 'Lrx2_uH', 'M1', 'M2', 'k1',
       'k2', 'Lmt_uH', 'Lmr1_uH', 'Lmr2_uH', 'Llt_uH', 'Llr1_uH', 'Llr2_uH',
       'Copperloss_Tx', 'Copperloss_Rx_1', 'Copperloss_Rx_2', 'Coreloss_W'],
      dtype='object')

In [27]:
B_field = pd.DataFrame(df_new.pop('__get_B_field').to_list())

In [28]:
B_field.columns=[
    'B_mean_leg_left', 'B_mean_leg_center', 'B_mean_leg_right',
    'B_mean_leg_top_left', 'B_mean_leg_bottom_left',
    'B_mean_leg_top_right', 'B_mean_leg_bottom_right'
]
df_new = df_new.join(B_field)
B_field

,B_mean_leg_left,B_mean_leg_center,B_mean_leg_right,B_mean_leg_top_left,B_mean_leg_bottom_left,B_mean_leg_top_right,B_mean_leg_bottom_right
0,0.069312,0.105800,0.104519,0.226288,0.225504,0.220188,0.220060
1,0.060373,0.051514,0.068563,0.169221,0.168974,0.177705,0.173988
2,0.062311,0.038665,0.083562,0.124133,0.120552,0.119516,0.122397
3,0.049074,0.048319,0.064469,0.238368,0.238431,0.234995,0.235081
4,0.033760,0.089453,0.044092,0.188241,0.188454,0.196190,0.195720
...,...,...,...,...,...,...,...
981,0.127822,0.115758,0.177624,0.180214,0.183056,0.180466,0.182302
982,0.030421,0.111180,0.045145,0.118866,0.118906,0.121912,0.120786
983,0.037359,0.078134,0.057970,0.039540,0.039426,0.040818,0.040595
984,0.038890,0.075477,0.060634,0.122064,0.122013,0.119818,0.118781


In [29]:


df_new.to_csv('parsed_data.csv', index=False)
df_new.head()

,uuid,w1,l1_leg,l1_top,l1_center,l2,h1,l2_tap,ratio,Tx_turns,...,Copperloss_Rx_1,Copperloss_Rx_2,Coreloss_W,B_mean_leg_left,B_mean_leg_center,B_mean_leg_right,B_mean_leg_top_left,B_mean_leg_bottom_left,B_mean_leg_top_right,B_mean_leg_bottom_right
0,6f06d2a2-6ed5-4c32-86a3-15ab68ea1040,10.4,10.6,4.6,18.8,22.4,34.0,0.0,0.51,14.0,...,2.631196,0.908127,9.10694,0.069312,0.105800,0.104519,0.226288,0.225504,0.220188,0.220060
1,f4293267-75da-4a1c-a66f-3549d607ac63,17.6,8.2,3.4,22.4,5.6,19.6,0.0,0.50,14.0,...,4.986744,1.218263,1.98987,0.060373,0.051514,0.068563,0.169221,0.168974,0.177705,0.173988
2,54d1a301-d86f-4aa2-91ed-c0634f8c11ad,26.0,4.6,3.4,21.2,14.0,43.6,0.0,0.51,14.0,...,1.564721,0.395008,2.72299,0.062311,0.038665,0.083562,0.124133,0.120552,0.119516,0.122397
3,68e1736f-ee54-4cb1-bf86-0618cfa951f3,20.0,8.2,2.2,21.2,23.6,26.8,0.0,0.50,14.0,...,2.197519,0.327247,7.77997,0.049074,0.048319,0.064469,0.238368,0.238431,0.234995,0.235081
4,e151eee3-84f9-43cf-a173-5fd9dfbc54e8,24.8,9.4,2.2,9.2,12.8,28.0,0.0,0.51,14.0,...,4.213384,0.633982,3.85803,0.033760,0.089453,0.044092,0.188241,0.188454,0.196190,0.195720
